In [6]:
import pandas as pd
import numpy as np
from src.recsys_baseline import VSKNN_STAN
from src.recsys_dev import rl_recommender
from src.evaluation import evaluate_sessions, MRR, HitRate, StrategicHybrid

# Grande Finalé

## Data loading

In [7]:
train_data = pd.read_csv(r'data\retailrocket\prepared\train-item-views_train_tr.txt',
                    sep='\t', nrows=100)


# Randomly fill the "event" column with values 0, 1, and 2
train_data['event'] = np.random.choice([0, 1, 2], size=len(train_data))

display(train_data.head())
display(train_data.shape)

,SessionId,Time,ItemId,Date,Datestamp,TimeO,ItemSupport,event
0,1,1.462752e+09,9654,2016-05-09,1.462752e+09,2016-05-09 00:01:15.848000+00:00,399,1
1,1,1.462752e+09,33043,2016-05-09,1.462752e+09,2016-05-09 00:02:53.912000+00:00,195,2
2,1,1.462752e+09,32118,2016-05-09,1.462752e+09,2016-05-09 00:04:03.569000+00:00,67,1
3,1,1.462752e+09,12352,2016-05-09,1.462752e+09,2016-05-09 00:05:29.870000+00:00,327,0
4,1,1.462752e+09,35077,2016-05-09,1.462752e+09,2016-05-09 00:06:30.072000+00:00,102,1


(100, 8)

In [8]:
test_data = pd.read_csv(r'data\retailrocket\prepared\train-item-views_test.txt',
                    sep='\t', nrows=40)

# Randomly fill the "event" column with values 0, 1, and 2
test_data['event'] = np.random.choice([0, 1, 2], size=len(test_data))

display(test_data.head())
display(test_data.shape)

,SessionId,Time,ItemId,Date,Datestamp,TimeO,ItemSupport,event
0,289,1.464221e+09,125013,2016-05-26,1.464221e+09,2016-05-26 00:00:18.301000+00:00,10,2
1,289,1.464222e+09,64068,2016-05-26,1.464221e+09,2016-05-26 00:14:07.735000+00:00,30,0
2,289,1.464222e+09,133346,2016-05-26,1.464221e+09,2016-05-26 00:14:38.934000+00:00,36,1
3,289,1.464222e+09,438457,2016-05-26,1.464221e+09,2016-05-26 00:18:34.305000+00:00,6,0
4,289,1.464222e+09,198930,2016-05-26,1.464221e+09,2016-05-26 00:18:48.607000+00:00,10,0


(40, 8)

## Hybrid model

In [9]:
# Initialize two recommenders that will be used in the hybrid model
model_HY1 = rl_recommender(
    n_history=10,
    reward_dict={0:1, 1:5, 2:10},

    # dqn
    mode = 'training',
    target_update_freq=5,
    num_episodes=100,
    log_to_wandb=True,
    dataset_name='retailrocket',    
)

model_HY2 = VSKNN_STAN(k=500,
                       sample_size=1000,
                       similarity='cosine',  
                       #stan:  
                       lambda_spw=7.24,  
                       lambda_snh=100,  
                       lambda_inh=3.62,  
                       #vsknn:  
                       lambda_ipw=3.62, 
                       lambda_idf=1
                       )

In [10]:
# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_HY1, model_HY2],
                        thresholds=[10]) # after 10 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train_data)

Started converting train data to tensors...
Episode tensors created successfully
Initializing gym environment...
Initializing offline training agent...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: meeslindhout (universityofappliedsciencesamsterdam). Use `wandb login --relogin` to force relogin


Started training loop...
Episode 1/100 completed with reward: 54
Episode 2/100 completed with reward: 10
Episode 3/100 completed with reward: 48
Episode 4/100 completed with reward: 10
Episode 5/100 completed with reward: 11
Episode 6/100 completed with reward: 6
Episode 7/100 completed with reward: 1
Episode 8/100 completed with reward: 36
Episode 9/100 completed with reward: 6
Episode 10/100 completed with reward: 53
Episode 11/100 completed with reward: 38
Episode 12/100 completed with reward: 35
Episode 13/100 completed with reward: 52
Episode 14/100 completed with reward: 15
Episode 15/100 completed with reward: 29
Episode 16/100 completed with reward: 22
Episode 17/100 completed with reward: 52
Episode 18/100 completed with reward: 54
Episode 19/100 completed with reward: 10
Episode 20/100 completed with reward: 48
Episode 21/100 completed with reward: 10
Episode 22/100 completed with reward: 11


In [ ]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test_data,
                  train_data=train_data,
                  cut_off=20)